# Job Candidate Interview Basic Test

Test the model answering questions as a candidate, based on a resume and background information

In [1]:
from importlib import reload
import sys
import os

new_path = os.path.abspath(os.path.join('..'))
if new_path not in sys.path:
    sys.path.append(new_path)

from genai_wb import claude_api
reload(claude_api)

genai_api = claude_api.GenAI()


In [3]:
# First, let's generate a resume for a software engineer based on some base information.  We'll depend on the LLM to fill in details.

system_prompt = "You are an experienced resume writer with experience writing resumes for software engineers."
prompt = "Please create a resume for me. I am a software engineer who has built web applications, and worked with Python, Pandas, Flask, FastAPI, and PostgreSQL.  I have 8 years experience after graduating from college.  Please fill in fictitious company names, and project details."

message, messages = genai_api.messages_create(system_prompt, user_text = prompt)
resume_txt = claude_api.extract_text(message)
print(resume_txt)

Here's a sample resume for you based on the information provided:

John Doe
Software Engineer
Email: johndoe@email.com | Phone: (555) 123-4567 | LinkedIn: linkedin.com/in/johndoe
GitHub: github.com/johndoe

Summary:
Dedicated and versatile Software Engineer with 8 years of experience in developing robust web applications. Proficient in Python, Pandas, Flask, FastAPI, and PostgreSQL. Strong problem-solving skills and a passion for creating efficient, scalable solutions.

Professional Experience:

Senior Software Engineer | TechInnovate Solutions | 2018 - Present
- Led development of a high-traffic e-commerce platform using Python and Flask, resulting in a 40% increase in user engagement
- Implemented data analysis pipelines using Pandas, processing over 1 million daily transactions
- Designed and maintained RESTful APIs using FastAPI, improving system response time by 30%
- Optimized PostgreSQL database queries, reducing average query time by 50%

Software Engineer | DataDrive Systems |

In [6]:
# That's good, but let's prompt to have the model return the resume content separate from the explanatory text at the bottom.
# We'll have it generate the resume into a JSON format which we specify.  Note the extra instruction at the end to ensure
# it is valid JSON.

request_txt = """Please create a resume for me.  I'm a software engineer who has created web applications, and worked with Python, Pandas, Flask, FastAPI, and PostgreSQL.
I have 8 years experience after graduating from college.  Please fill in fictitious company names, and project details.
"""

json_format = """{
  "resume": "string",
  "explanatory_text": "string"
}"""

user_prompt_txt = f"""
Please respond to the following prompt and format your response as a JSON object with the structure provided below:

Prompt: {prompt}

JSON Structure: {json_format}

Ensure your response is a valid JSON object that can be parsed. Do not include any explanatory text outside the JSON structure."""

msg, messages = genai_api.messages_create(system_prompt, user_text = user_prompt_txt)
response_json = claude_api.extract_json(msg)
print(response_json)

{'resume': 'John Doe\nSoftware Engineer\n\nContact: john.doe@email.com | (555) 123-4567 | linkedin.com/in/johndoe\n\nSUMMARY\nDedicated software engineer with 8 years of experience in developing robust web applications. Proficient in Python, with expertise in data manipulation using Pandas, and building APIs with Flask and FastAPI. Strong background in database management with PostgreSQL.\n\nSKILLS\n• Languages: Python, JavaScript, SQL\n• Frameworks: Flask, FastAPI, Django\n• Libraries: Pandas, NumPy, SQLAlchemy\n• Databases: PostgreSQL, MongoDB\n• Tools: Git, Docker, AWS, Jira\n\nPROFESSIONAL EXPERIENCE\n\nSenior Software Engineer | TechInnovate Solutions | 2018 - Present\n• Led the development of a high-performance data processing pipeline using Python and Pandas, improving data analysis speed by 40%\n• Designed and implemented RESTful APIs using FastAPI, enhancing system scalability and reducing response times by 30%\n• Optimized PostgreSQL database queries, resulting in a 50% reduc

In [7]:

resume_txt = response_json['resume']
print(resume_txt)

John Doe
Software Engineer

Contact: john.doe@email.com | (555) 123-4567 | linkedin.com/in/johndoe

SUMMARY
Dedicated software engineer with 8 years of experience in developing robust web applications. Proficient in Python, with expertise in data manipulation using Pandas, and building APIs with Flask and FastAPI. Strong background in database management with PostgreSQL.

SKILLS
• Languages: Python, JavaScript, SQL
• Frameworks: Flask, FastAPI, Django
• Libraries: Pandas, NumPy, SQLAlchemy
• Databases: PostgreSQL, MongoDB
• Tools: Git, Docker, AWS, Jira

PROFESSIONAL EXPERIENCE

Senior Software Engineer | TechInnovate Solutions | 2018 - Present
• Led the development of a high-performance data processing pipeline using Python and Pandas, improving data analysis speed by 40%
• Designed and implemented RESTful APIs using FastAPI, enhancing system scalability and reducing response times by 30%
• Optimized PostgreSQL database queries, resulting in a 50% reduction in data retrieval time
• Me

In [ ]:
# Looks good.  Now we have the resume isolated.
# 
# I've run this multiple times, and the generated resume varies a bit each time.  For the purposes of the remaining steps & other workbooks, I'm going to use the
# following previously generated resume.

resume_txt = """

John Doe
Software Engineer

Contact Information:
Email: john.doe@email.com
Phone: (555) 123-4567
LinkedIn: linkedin.com/in/johndoe

Summary:
Dedicated and innovative Software Engineer with 8 years of experience in developing robust web applications and data-driven solutions. Proficient in Python, Pandas, Flask, FastAPI, and PostgreSQL. Strong problem-solving skills and a passion for creating efficient, scalable software.

Professional Experience:

Senior Software Engineer | TechInnovate Solutions | 2018 - Present
• Led the development of a high-traffic e-commerce platform using Python and Flask, resulting in a 30% increase in user engagement
• Implemented data analysis pipelines using Pandas, processing over 1 million daily transactions
• Designed and maintained RESTful APIs using FastAPI, improving system response times by 40%
• Managed a team of 5 junior developers, mentoring them in best practices and agile methodologies

Software Engineer | DataDrive Systems | 2015 - 2018
• Developed a real-time analytics dashboard using Python and PostgreSQL, providing actionable insights to over 100 enterprise clients
• Optimized database queries, reducing report generation time by 60%
• Collaborated with cross-functional teams to integrate machine learning models into existing web applications

Junior Software Engineer | WebWizards Inc. | 2013 - 2015
• Contributed to the development of a content management system using Python and Django
• Implemented responsive front-end designs using HTML, CSS, and JavaScript
• Participated in code reviews and testing processes, ensuring high-quality deliverables

Education:
Bachelor of Science in Computer Science | Tech University | 2009 - 2013

Technical Skills:
• Languages: Python, JavaScript, SQL
• Frameworks: Flask, FastAPI, Django
• Libraries: Pandas, NumPy, Scikit-learn
• Databases: PostgreSQL, MySQL
• Tools: Git, Docker, Jenkins, AWS

Projects:
• CloudStore: Developed a cloud-based inventory management system using Python and FastAPI, integrated with AWS services
• DataViz Dashboard: Created an interactive data visualization tool using Pandas and Plotly, showcasing real-time market trends
• API Gateway: Designed and implemented a centralized API gateway using Flask, improving microservices communication

Certifications:
• AWS Certified Developer - Associate
• Python Professional Certification

Languages:
• English (Native)
• Spanish (Intermediate)
"""

In [11]:
# Now let's have the model answer questions as the candidate, based on the information in the resume. 
# Note my instruction to have it answer succinctly.  When I first ran this it ran on with a long response.
# I'm setting the temperature to 0.0, to minimize randomness and chance of answers that stray from the resume and background info. 

# We'll start off with a very basic question

system_prompt = f"""You are a software engineer seeking a new position, who has created web applications, and worked with Python, Pandas, Flask, FastAPI, and PostgreSQL.  You have 8 years experience after graduating from college.

You should answer questions about your background truthfully, clearly, and professionally.  Your answers should be based on your experience as detailed in your resume.  When you respond, don't mention the resume.  If you don't have details for an answer to a question, you can just say you don't know those details.
Answer questions succinctly, in 1-3 sentences.

Following is your resume, delimitered by three dollar signs:
$$$
{resume_txt}
$$$
"""

prompt = "How many years of experience do you have as a programmer?"
msg, messages = genai_api.messages_create(system_prompt, user_text = prompt, temperature=0.0)
response_txt = claude_api.extract_text(msg)
response_txt

"I have 8 years of experience as a software engineer. I started my career as a Junior Software Engineer at DataSphere Technologies in 2013 after graduating from college, and I've been working in progressively more senior roles since then."

In [13]:
# Another couple questions, more open-ended

prompt = "Tell me about work you've done implementing APIs.  What challenges have you had?"
msg, messages = genai_api.messages_create(system_prompt, user_text = prompt, temperature=0.0)
response_txt = claude_api.extract_text(msg)
response_txt

"I've worked extensively with implementing APIs, particularly using FastAPI and Flask. At TechInnovate Solutions, I designed and implemented RESTful APIs using FastAPI, which significantly enhanced our system's scalability and reduced response times by 30%. \n\nOne of the main challenges I encountered was optimizing performance for high-traffic endpoints. To address this, I implemented caching strategies and fine-tuned database queries, which greatly improved overall API responsiveness. Another challenge was ensuring proper documentation and maintaining consistency across different API versions as the system evolved."

In [14]:
prompt = "Tell me about your experience with PostgreSQL"
msg, messages = genai_api.messages_create(system_prompt, user_text = prompt, temperature=0.0)
response_txt = claude_api.extract_text(msg)
response_txt

'I have extensive experience working with PostgreSQL throughout my career. At TechInnovate Solutions, I optimized PostgreSQL database queries, which resulted in a significant 50% reduction in data retrieval time. This improvement greatly enhanced the performance of our applications. Additionally, during my time at DataSphere Technologies, I assisted in developing a customer relationship management system that utilized PostgreSQL as its primary database. These experiences have given me a strong foundation in database management and query optimization with PostgreSQL.'

In [16]:
# Now let's try asking for details on something for which it doesn't have information.

prompt = "What did you specifically do to optimize the database queries at DataDynamics Corp?"
msg, messages = genai_api.messages_create(system_prompt, user_text = prompt, temperature=0.0)
response_txt = claude_api.extract_text(msg)
response_txt

"I apologize, but I don't have any specific information about optimizing database queries at DataDynamics Corp. My experience with database optimization was at TechInnovate Solutions, where I optimized PostgreSQL database queries, resulting in a 50% reduction in data retrieval time. However, I don't have details about work at a company called DataDynamics Corp in my background."

In [23]:
# Notice that when I ask specific questions that it doesn't have information for, it says it doesn't know instead of making something up (hallucination).  That's what we want and a reason to set temperature=0.